<a href="https://colab.research.google.com/github/EunkuBae/small_scripts_and_tkinter_examples/blob/master/Copy_of_naver_review_classifications_pytorch_kobert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3
!pip install torch

     |████████████████████████████████| 54.7MB 61kB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 348kB 21.5MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595688 sha256=af21f0c2f86fa2612916504f445eec24eff688de8fea0b40c07a717fd09079cf
  Stored in directory: /root/.cache/pip/wheels/37/65/52/63032864a0f31a08b9a88569f803b5bafac8abd207fd7f7534
Successfully built gluonnlp
     |████████████████████████████████| 1.2MB 19.0MB/s 
     |████████████████████████████████| 757kB 16.4MB/s 
     |████████████████████████████████| 890kB 57.4MB/s 
     |████████████████████████████████| 3.0MB 55.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=c660208513ab2cb84bf0495a5586121c1087ada403c1f5027c44279a439401b9
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a5

In [3]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-34fwy09h
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-34fwy09h
  Created wheel for kobert: filename=kobert-0.1.2-cp37-none-any.whl size=12708 sha256=e40c4f2625d20e89e016c6025278d3c925f815eef96d88f08d0aa91687f32ff0
  Stored in directory: /tmp/pip-ephem-wheel-cache-6gjge01x/wheels/a2/b0/41/435ee4e918f91918be41529283c5ff86cd010f02e7525aecf3
Successfully built kobert


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [6]:
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [7]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [8]:
##GPU 사용 시
device = torch.device("cuda:0")
# device = torch.device("cpu")

In [9]:
bertmodel, vocab = get_pytorch_kobert_model()

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]


In [10]:
!wget https://www.dropbox.com/s/374ftkec978br3d/ratings_train.txt?dl=1
!wget https://www.dropbox.com/s/977gbwh542gdy94/ratings_test.txt?dl=1

--2021-03-16 21:32:48--  https://www.dropbox.com/s/374ftkec978br3d/ratings_train.txt?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.7.18, 2620:100:601c:18::a27d:612
Connecting to www.dropbox.com (www.dropbox.com)|162.125.7.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/374ftkec978br3d/ratings_train.txt [following]
--2021-03-16 21:32:48--  https://www.dropbox.com/s/dl/374ftkec978br3d/ratings_train.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc5164395fdb6f732656b60c2726.dl.dropboxusercontent.com/cd/0/get/BK3XvT7l-KTIz5tI5kAOhVPUKNVOspW4JVSk0hb7cAdesfTJ2jcdgC7O3FbtqI65NQIybbbETZKHTubmdhdNBUuZV9KVXVLKdzv6syP3RZuYvw7IKFIQndN_ehqNvy6qJt0CCVromd002g9GjyABDWVV/file?dl=1# [following]
--2021-03-16 21:32:49--  https://uc5164395fdb6f732656b60c2726.dl.dropboxusercontent.com/cd/0/get/BK3XvT7l-KTIz5tI5kAOhVPUKNVOspW4JVSk0hb7cAdesfTJ2jcdgC7O3FbtqI65NQIybbbET

In [11]:
dataset_train = nlp.data.TSVDataset("ratings_train.txt?dl=1", field_indices=[1,2], num_discard_samples=1)
dataset_test = nlp.data.TSVDataset("ratings_test.txt?dl=1", field_indices=[1,2], num_discard_samples=1)

In [12]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [13]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))


In [14]:
## Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [15]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [16]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [17]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [18]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [19]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [20]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [21]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [22]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [23]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [24]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 0.7351676821708679 train acc 0.4375
epoch 1 batch id 201 loss 0.4979616403579712 train acc 0.5696517412935324
epoch 1 batch id 401 loss 0.3867872655391693 train acc 0.6818110972568578
epoch 1 batch id 601 loss 0.4586159288883209 train acc 0.7329190931780366
epoch 1 batch id 801 loss 0.4436682164669037 train acc 0.7622893258426966
epoch 1 batch id 1001 loss 0.297607958316803 train acc 0.7799388111888111
epoch 1 batch id 1201 loss 0.3364447355270386 train acc 0.7928028726061616
epoch 1 batch id 1401 loss 0.40781208872795105 train acc 0.8017822091363311
epoch 1 batch id 1601 loss 0.33684027194976807 train acc 0.8094745471580262
epoch 1 batch id 1801 loss 0.21638619899749756 train acc 0.815814131038312
epoch 1 batch id 2001 loss 0.2957002818584442 train acc 0.8216907171414293
epoch 1 batch id 2201 loss 0.2537045478820801 train acc 0.8266767946388005

epoch 1 train acc 0.8298759243458476


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



epoch 1 test acc 0.8847306585677749


...


epoch 3 test acc 0.8978580562659847


epoch 4 batch id 1 loss 0.46907058358192444 train acc 0.875
epoch 4 batch id 201 loss 0.05691150203347206 train acc 0.9553793532338308
epoch 4 batch id 401 loss 0.09670975059270859 train acc 0.9561253117206983
epoch 4 batch id 601 loss 0.23882615566253662 train acc 0.95801268718802
epoch 4 batch id 801 loss 0.21974384784698486 train acc 0.9597183208489388
epoch 4 batch id 1001 loss 0.17204248905181885 train acc 0.9609609140859141
epoch 4 batch id 1201 loss 0.1543705016374588 train acc 0.962023834304746
epoch 4 batch id 1401 loss 0.0507860891520977 train acc 0.9629505710206995
epoch 4 batch id 1601 loss 0.07417634129524231 train acc 0.9638214397251718
epoch 4 batch id 1801 loss 0.034040339291095734 train acc 0.9648892976124376
epoch 4 batch id 2001 loss 0.03825967386364937 train acc 0.9656812218890555
epoch 4 batch id 2201 loss 0.12626144289970398 train acc 0.9658890845070423

epoch 4 train acc 0.9665213488339021



epoch 4 test acc 0.8975383631713555


epoch 5 batch id 1 loss 0.5409494638442993 train acc 0.875
epoch 5 batch id 201 loss 0.0516759492456913 train acc 0.9731032338308457
epoch 5 batch id 401 loss 0.027255859225988388 train acc 0.974283042394015
epoch 5 batch id 601 loss 0.1425936073064804 train acc 0.9749116056572379
epoch 5 batch id 801 loss 0.11753714084625244 train acc 0.9754213483146067
epoch 5 batch id 1001 loss 0.1308121234178543 train acc 0.9763517732267732
epoch 5 batch id 1201 loss 0.014514860697090626 train acc 0.9769723147377186
epoch 5 batch id 1401 loss 0.043295346200466156 train acc 0.9773487687366167
epoch 5 batch id 1601 loss 0.04376139119267464 train acc 0.9775628513429107
epoch 5 batch id 1801 loss 0.0553450845181942 train acc 0.9779896585230428
epoch 5 batch id 2001 loss 0.008572905324399471 train acc 0.978401424287856
epoch 5 batch id 2201 loss 0.05013657361268997 train acc 0.9784685938209905

epoch 5 train acc 0.9785600647042093



epoch 5 test acc 0.8972786125319693
